In [2]:
import As_Romenia_Toda as art

In [3]:
import random as rnd

rnd.seed(89)
(estado_inicial, estado_objetivo) = rnd.sample(art.estados_romenia, 2)

print(f"{estado_inicial['estado']} e {estado_objetivo['estado']}")

Timisoara e Craiova
